# Feature importance

In [14]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [34]:
data = pd.read_csv(r'../data/dataset_merged_with_price.csv', index_col=0)
data.head()

,active_addresses,tweets,top_100_percent,difficulty,transactions,av_transaction_size,market_cap,confirmation_time,median_transaction_value,send_usd,...,av_transaction_value,full_name,coin,FEDFUNDS,gold_open,gold_high,gold_low,gold_close,gold_volume,btc_price
date,,,,,,,,,,,,,,,,,,,,,
2015-08-01,267062.0,26960.0,20.93,5.227830e+10,147326.0,NaN,4.006359e+09,9.29,53.55,219635191.0,...,1502.0,bitcoin,btc,0.14,NaN,NaN,NaN,NaN,0,280.98
2015-08-02,260292.0,24294.0,20.93,5.227830e+10,103449.0,NaN,4.009289e+09,9.54,48.11,180286651.0,...,1827.0,bitcoin,btc,0.14,NaN,NaN,NaN,NaN,0,280.32
2015-08-03,247772.0,35036.0,20.95,5.227830e+10,119687.0,NaN,4.040683e+09,9.17,72.78,289689678.0,...,2436.0,bitcoin,btc,0.14,1095.72,1097.68,1085.79,1086.66,78495,282.59
2015-08-04,260149.0,34486.0,20.83,5.227830e+10,132411.0,NaN,4.044523e+09,10.59,129.19,332340266.0,...,2510.0,bitcoin,btc,0.14,1085.32,1094.78,1080.94,1087.61,85634,283.59
2015-08-05,280613.0,31470.0,20.77,5.227830e+10,133625.0,NaN,4.022246e+09,10.83,123.22,347586782.0,...,2627.0,bitcoin,btc,0.14,1087.31,1092.48,1082.92,1084.77,85788,282.59


Analyse the feature importance: 

In [57]:
X = data[["tweets", "transactions", "google_trends", "FEDFUNDS", "gold_open","gold_high","gold_low","gold_close"]].dropna()
print(X)

             tweets  transactions  google_trends  FEDFUNDS  gold_open  \
date                                                                    
2015-08-03  35036.0      119687.0           7.80      0.14    1095.72   
2015-08-04  34486.0      132411.0           7.48      0.14    1085.32   
2015-08-05  31470.0      133625.0           7.27      0.14    1087.31   
2015-08-06  35658.0      134695.0           6.63      0.14    1084.82   
2015-08-07  47467.0      111208.0           6.84      0.14    1089.77   
...             ...           ...            ...       ...        ...   
2020-12-23  43942.0      338309.0           0.00      0.09    1861.01   
2020-12-24  42618.0      269433.0           0.00      0.09    1873.39   
2020-12-25  49739.0      303757.0           0.00      0.09    1873.39   
2020-12-26  72172.0      273817.0           0.00      0.09    1873.39   
2020-12-27  88788.0      308857.0           0.00      0.09    1873.39   

            gold_high  gold_low  gold_close  
date

In [59]:
Y = data["btc_price"].iloc[2:].to_frame()
print(Y)

            btc_price
date                 
2015-08-03     282.59
2015-08-04     283.59
2015-08-05     282.59
2015-08-06     279.11
2015-08-07     278.09
...               ...
2020-12-23   23562.00
2020-12-24   23307.00
2020-12-25   24013.00
2020-12-26   25252.00
2020-12-27   27061.00

[1974 rows x 1 columns]


In [61]:
Y = data["btc_price"].iloc[2:]
Y = Y.fillna(Y.median())

#model = ExtraTreesClassifier()
#model.fit(X,Y)
#print (model.feature_importances_)
#feat_importances = pd.Series(model.feature_importances_, index=X.columns)
#feat_importances.nlargest(5).plot(kind="barh")
#plt.show()